In [3]:
import dash
from dash import dcc, html, Input, Output
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests

In [4]:
def lv95ToWgs84(x,y):
    """
    Convert swiss coordinates (LV03) to latitude, longitude values (WGS84)
    https://www.swisstopo.admin.ch › ch1903wgs84_d

    Parameters
    ----------
    x : float
        lv95 x coordinate
    y : float
        lv95 y coordinate

    Result
    ---------
    lat : float
        latitude
    lon : float
        longitude

    """
    ys=(x-2600000)/1000000 #LV95
    xs=(y-1200000)/1000000  #LV95
    lon=2.6779094+4.728982*ys+0.791484*ys*xs+0.1306*ys*xs*xs-0.0436*ys*ys*ys
    lat=16.9023892+3.238272*xs-0.270978*ys*ys-0.002528*xs*xs-0.0447*ys*ys*xs-0.0140*xs*xs*xs
    lon=lon*100/36
    lat=lat*100/36
    if np.isnan(lon) or np.isnan(lat):
        return None
    else:
        return (lat,lon)

def get_coordinates(adress):

    url_1 = "https://api3.geo.admin.ch//rest/services/api/SearchServer?lang=de&searchText="+adress+"&type=locations&sr=2056"
    response = requests.get(url_1).json()
    try:
        y = response['results'][0]['attrs']['y']
        x = response['results'][0]['attrs']['x']
        coord = str(y)+","+str(x)
    except:
        print("Error loading Adress")
    lat, lon = lv95ToWgs84(y,x)

    return coord,lat,lon

def get_building_id(coord):

    url_2= "https://api3.geo.admin.ch//rest/services/api/MapServer/identify?geometryType=esriGeometryPoint&returnGeometry=true&layers=all:ch.bfe.solarenergie-eignung-daecher&geometry="+coord+"&tolerance=0&order=distance&lang=de&sr=2056"
    response = requests.get(url_2).json()
    try:
        building_id = response['results'][0]['attributes']['building_id']
    except:
        print("Error loading coordinates"+coord)

    return building_id

def get_sonnendach(building_id): 
    feature_ids=[]
    pv_class=[]
    ausrichtung=[]
    pv_yield=[]
    neigung=[]
    area=[]
    building_ids=[]

    url_3 = "https://api3.geo.admin.ch//rest/services/api/MapServer/find?layer=ch.bfe.solarenergie-eignung-daecher&searchField=building_id&searchText="+str(building_id)+"&contains=false"
    response = requests.get(url_3).json()
    
    for idx, num in enumerate(response['results']):
        feature_id = response['results'][idx]['featureId']   
        if feature_id not in feature_ids:
            feature_ids.append(feature_id)
            building_ids.append(building_id)
            pv_class.append(response['results'][idx]['attributes']['klasse'])
            ausrichtung.append(response['results'][idx]['attributes']['ausrichtung'])
            pv_yield.append(response['results'][idx]['attributes']['stromertrag'])
            neigung.append(response['results'][idx]['attributes']['neigung'])
            area.append(response['results'][idx]['attributes']['flaeche'])

    df = pd.DataFrame([building_ids,feature_ids,pv_class,ausrichtung,pv_yield,neigung,area]).transpose()
    df.columns=['building_ids','feature_ids','PV_klasse','ausrichtung','Stromertrag','neigung','area']
    return df


In [5]:
adress = 'Dachlissen'+"%20"+str(100)+"%20"+str(8932)+"%20"+'Mettmenstetten'
coord,lat,lon = get_coordinates(adress)
building_id = get_building_id(coord)
df = get_sonnendach(building_id)

In [6]:
df

,building_ids,feature_ids,PV_klasse,ausrichtung,Stromertrag,neigung,area
0,725140.0,11627701.0,5.0,18.0,25925.0,39.0,135.194607
1,725140.0,11627700.0,1.0,-162.0,13066.0,38.0,131.073914


In [6]:
list(df.feature_ids)

[11627701.0, 11627700.0]

In [43]:
roof_info[11627700]
    

building_ids    7.251400e+05
feature_ids     1.162770e+07
PV_klasse       1.000000e+00
ausrichtung    -1.620000e+02
Stromertrag     1.306600e+04
neigung         3.800000e+01
area            1.310739e+02
Name: 1, dtype: float64

In [42]:
roof_info

{11627701.0: building_ids    7.251400e+05
 feature_ids     1.162770e+07
 PV_klasse       5.000000e+00
 ausrichtung     1.800000e+01
 Stromertrag     2.592500e+04
 neigung         3.900000e+01
 area            1.351946e+02
 Name: 0, dtype: float64,
 11627700.0: building_ids    7.251400e+05
 feature_ids     1.162770e+07
 PV_klasse       1.000000e+00
 ausrichtung    -1.620000e+02
 Stromertrag     1.306600e+04
 neigung         3.800000e+01
 area            1.310739e+02
 Name: 1, dtype: float64}